In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# from sklearn import SimpleImputer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

/kaggle/input/playground-series-s3e26/sample_submission.csv
/kaggle/input/playground-series-s3e26/train.csv
/kaggle/input/playground-series-s3e26/test.csv


In [2]:
# !pip install --upgrade scikit-learn

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s3e26/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e26/test.csv')


In [4]:
train.head()


,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [5]:
test.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7905,3839,D-penicillamine,19724,F,N,Y,N,N,1.2,546.0,3.37,65.0,1636.0,151.90,90.0,430.0,10.6,2.0
1,7906,2468,D-penicillamine,14975,F,N,N,N,N,1.1,660.0,4.22,94.0,1257.0,151.90,155.0,227.0,10.0,2.0
2,7907,51,Placebo,13149,F,N,Y,N,Y,2.0,151.0,2.96,46.0,961.0,69.75,101.0,213.0,13.0,4.0
3,7908,2330,D-penicillamine,20510,F,N,N,N,N,0.6,293.0,3.85,40.0,554.0,125.55,56.0,270.0,10.6,2.0
4,7909,1615,D-penicillamine,21904,F,N,Y,N,N,1.4,277.0,2.97,121.0,1110.0,125.00,126.0,221.0,9.8,1.0


In [6]:
print("Dataset Info:")
print(train.info())

# Display summary statistics of numerical columns
print("\nSummary Statistics:")
print(train.describe())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             7905 non-null   int64  
 1   N_Days         7905 non-null   int64  
 2   Drug           7905 non-null   object 
 3   Age            7905 non-null   int64  
 4   Sex            7905 non-null   object 
 5   Ascites        7905 non-null   object 
 6   Hepatomegaly   7905 non-null   object 
 7   Spiders        7905 non-null   object 
 8   Edema          7905 non-null   object 
 9   Bilirubin      7905 non-null   float64
 10  Cholesterol    7905 non-null   float64
 11  Albumin        7905 non-null   float64
 12  Copper         7905 non-null   float64
 13  Alk_Phos       7905 non-null   float64
 14  SGOT           7905 non-null   float64
 15  Tryglicerides  7905 non-null   float64
 16  Platelets      7905 non-null   float64
 17  Prothrombin    7905 non-null   float64

In [7]:
categorical_columns = ['Drug', 'Sex','Ascites','Hepatomegaly','Spiders','Edema']
# df_encoded = pd.get_dummies(train, columns=['Drug', 'Sex','Ascites','Hepatomegaly','Spiders','Edema'], prefix=['Drug_hot', 'Sex_hot','Ascites_hot','Hepatomegaly_hot','Spiders_hot','Edema_hot'])


In [8]:
missing_values_train = train.isnull().sum()
print("\nMissing Values:")
print(missing_values_train)



Missing Values:
id               0
N_Days           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
Status           0
dtype: int64


In [9]:
missing_values_test = test.isnull().sum()
print("\nMissing Values:")
print(missing_values_test)


Missing Values:
id               0
N_Days           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
dtype: int64


In [10]:
train['Status'] = train['Status'].map({'C': 0, 'D': 2, 'CL': 1})
X = train.drop(columns = ['Status','id','N_Days'])
y = train['Status']
print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

0       2
1       0
2       2
3       0
4       0
       ..
7900    0
7901    0
7902    2
7903    2
7904    0
Name: Status, Length: 7905, dtype: int64


In [11]:
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_columns)
    ],
    remainder='passthrough'
)

In [12]:
# classifier1 = XGBClassifier(n_estimators=100, random_state=42)
# classifier2 = XGBClassifier(n_estimators=200, random_state=42)

# # Create the ensemble model
# ensemble_model = VotingClassifier(estimators=[
#     ('xgb1', classifier1),
#     ('xgb2', classifier2)
# ], voting='soft')

rf_model = RandomForestClassifier(random_state=42)
gb_model = GradientBoostingClassifier(random_state=42)

# Create an ensemble model using a voting classifier
ensemble_model = VotingClassifier(estimators=[
    ('RandomForest', rf_model),
    ('GradientBoosting', gb_model)
], voting='soft')  # 'soft' enables predicted probabilities for log_loss


# # Create the full pipeline
# pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                              ('ensemble', ensemble_model)])
# classifier_nb = GaussianNB()
# classifier_xgb = XGBClassifier(n_estimators=150, random_state=42)

# # Create the ensemble model
# ensemble_model = VotingClassifier(estimators=[
#     ('nb', classifier_nb),
#     ('xgb', classifier_xgb)
# ], voting='soft')  # 'soft' enables probability voting
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('ensemble', ensemble_model)])


In [13]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot', OneHotEncoder(),
                                                  ['Drug', 'Sex', 'Ascites',
                                                   'Hepatomegaly', 'Spiders',
                                                   'Edema'])])),
                ('ensemble',
                 VotingClassifier(estimators=[('RandomForest',
                                               RandomForestClassifier(random_state=42)),
                                              ('GradientBoosting',
                                               GradientBoostingClassifier(random_state=42))],
                                  voting='soft'))])

In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
accuracy = accuracy_score(y_test, y_pred)
# classification_report_result = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
# print("Classification Report:")
# print(classification_report_result)

Accuracy: 0.8216694772344013


In [16]:
print(y_test)

3467    0
465     0
453     0
4913    2
5907    0
       ..
2921    2
4614    2
6861    0
4105    2
4102    0
Name: Status, Length: 2372, dtype: int64


In [17]:
# y_pred = pipeline.predict(test)

In [18]:
test_probs = pipeline.predict_proba(X_test)

In [19]:
print(test_probs)

[[0.95598303 0.00822418 0.03579279]
 [0.50494797 0.05932861 0.43572343]
 [0.84899315 0.01386715 0.13713971]
 ...
 [0.53093531 0.03261081 0.43645387]
 [0.10213761 0.05587922 0.84198316]
 [0.98001922 0.0014388  0.01854199]]


In [20]:
loss = log_loss(y_test, test_probs)

print(f'Log Loss: {loss:.4f}')

Log Loss: 0.4687


In [21]:
# preds = pd.DataFrame(test_probs, columns = ['Status_C', 'Status_CL', 'Status_D'])
# sub['Status_C'] = preds['Status_C']
# sub['Status_CL'] = preds['Status_CL']
# sub['Status_D'] = preds['Status_D']
# sub.to_csv('submission.csv', index=False)
# print()
# print(sub.head(5))
# print()
# print("Submission file generated...")
# print()
# print("You've earned yourself a coffee...")

In [22]:
output_df = pd.read_csv('/kaggle/input/playground-series-s3e26/sample_submission.csv')
output_df.head()

,id,Status_C,Status_CL,Status_D
0,7905,0.628084,0.034788,0.337128
1,7906,0.628084,0.034788,0.337128
2,7907,0.628084,0.034788,0.337128
3,7908,0.628084,0.034788,0.337128
4,7909,0.628084,0.034788,0.337128


In [23]:
y_pred = pipeline.predict_proba(test)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ['Status_C', 'Status_CL','Status_D']
y_pred.head()

,Status_C,Status_CL,Status_D
0,0.668286,0.050675,0.281039
1,0.660153,0.145703,0.194144
2,0.147207,0.025336,0.827457
3,0.922604,0.012201,0.065195
4,0.639711,0.038606,0.321683


In [24]:
ids = test['id']
submission_df = pd.DataFrame()
submission_df = y_pred  
submission_df['id'] = ids
submission_df = 0.5*submission_df + 0.5*output_df
submission_df['id'] = submission_df['id'].apply(lambda x : int(x))
submission_df.head()

,Status_C,Status_CL,Status_D,id
0,0.648185,0.042732,0.309083,7905
1,0.644118,0.090245,0.265636,7906
2,0.387645,0.030062,0.582293,7907
3,0.775344,0.023495,0.201161,7908
4,0.633898,0.036697,0.329406,7909


In [25]:
column_name = 'id'

# Pop the column from its current position (index 3)
column_to_move = submission_df.pop(column_name)

# Insert the column at the desired position (index 0)
submission_df.insert(0, column_name, column_to_move)

In [26]:
submission_df.head()

,id,Status_C,Status_CL,Status_D
0,7905,0.648185,0.042732,0.309083
1,7906,0.644118,0.090245,0.265636
2,7907,0.387645,0.030062,0.582293
3,7908,0.775344,0.023495,0.201161
4,7909,0.633898,0.036697,0.329406


In [27]:
submission_df.to_csv('submission.csv', index= False)